In [1]:
import pandas as pd

In [2]:
instance = 'c2'
v = 25
c = 200
n = 20
depot = 0

In [3]:
dados = pd.read_csv(instance+".txt",index_col=0,sep=';').to_dict()

In [4]:
distancia = [[0 for _ in range(n)] for _ in range(n)]

In [5]:
def dist(i,j):
    x = (dados['x'][i]-dados['x'][j])**2
    y = (dados['y'][i]-dados['y'][j])**2
    
    return (x+y)**(1/2)

In [6]:
comeco_horizonte = dados['tempo_preparo'][depot]
fim_horizonte = dados['tempo_entrega'][depot]

In [7]:
M = -1
for i in range(n):
    for j in range(n):
        if i != j:
            distancia[i][j]=round(dist(i,j)) 
            M = max(M,dados['tempo_entrega'][i]+distancia[i][j]-dados['tempo_preparo'][i])
        else: 
            distancia[i][j]=float('inf')

In [8]:
from docplex.mp.model import Model
from docplex.mp.linear import LinearExpr

In [9]:
model = Model("vrptw")

### Variáveis

In [10]:
travels = model.binary_var_cube(v,n,n,'travel')#matriz tridimensional k*v²

In [11]:
service = {(k,i): model.continuous_var(
    lb=dados['tempo_preparo'][i],
        ub=dados['tempo_entrega'][i],name=f'service_start_{k}_{i}') 
           for k in range(v) for i in range(n)}#momento em que o serviço começa no cliente i

### Função Objetiva

In [12]:
model.minimize(model.sum(travels[k,i,j]*distancia[i][j] for i in range(n) for j in range(n) for k in range(v)))

### Restrições

In [13]:
#Visitar o cliente i é obrigatório
for i in range(1,n):
    model.add_constraint(
     model.sum(travels[k,i,j]
       for k in range(v)
           for j in range(n))==1,
            'Visit_Client_'+str(i))

In [14]:
#Respeitar Capacidade
for k in range(v):
    model.add_constraint(
        model.sum(travels[k,i,j]*dados['demanda'][i]
            for i in range(n)
                for j in range(n)) <= c,
                    'Capacity_Vehicle_'+str(k))

In [15]:
#Saída a Partir do Depósito
for k in range(v):
    model.add_constraint(
        model.sum((n*n)*travels[k,0,j]
            for j in range(1,n)) - 
                model.sum(travels[k,i,j] 
                      for i in range(1,n) 
                          for j in range(1,n)) >= 0,
                'Exit_Depot_Vehicle_'+str(k))

In [16]:
#Conservação de Fluxo
for k in range(v):
    for i in range(n):
        model.add_constraint(
            model.sum(travels[k,i,j]-travels[k,j,i]
                    for j in range(n)) == 0,
                        f'Flux_Conservation_Vehicle_{k}_Node_{i}')

In [17]:
#Tempo de Saida
for k in range(v):
    for i in range(n):
        for j in range(i+1,n):
            model.add_constraint(service[k,i]+distancia[i][j]-M*(1-travels[k,i,j]) <= service[k,j],
                                f'Exit_Time_{k}_{i}_{j}')

In [18]:
#Forbidden Arcs
model.add_constraint(model.sum(travels[k,i,i] for k in range(v) for i in range(n))==0);

In [19]:
import time
start = time.time() #Tempo de ínicio
subcicle = True     #Condição de parada
#Assumindo que existe subciclos, nós...
while subcicle == True:

    #Rota percorrida por cada veículo
    routes = {x:{} for x in range(v)}    

    #Solução VRP - Dicionário Desordenado
    _Solution = model.solve(log_output=True).as_dict()
    #saida veículo
    #    |   |
    #    v   v
    #AArc_k_i_j: True/False <-Valores Possíveis
    #      ^
    #      |
    #   chegada
    #
    #AArc_k_i_j = Chave

    for i in _Solution:
        if i.name[0] != "t":
            continue
        #
        #
        #"Arc_k_i_j"
        #     |   
        #     v                      0   1   2
        aux = i.name.split("_")[1:] #<- ["k","i","j"]
        #        ^
        #        |
        #["Arc","i","j","k"]
        #
        #         veiculo=k    saida=i      chegada=j
        #             |            |            |
        #             v            v            v
        routes[int(aux[0])][int(aux[1])]=int(aux[2])
        # ^
        # |_ Rotas agrupadas por veículo

    print(routes)
    count = 0 #Contador para as eliminações de subciclo
    summ = 0 #Condição de parada

    #O trecho de código abaixo deve remover o caminho
    #principal para futuramente verificarmos se existe
    #algum subciclo na rota do veículo 'k'
    for k in range(v):

        #Primeiramente, verificamos se o veículo 'k'
        #percorreu, ou não, pelo menos um arco
        if len(routes[k]) > 0:

            #Se ele percorreu pelo menos um arco:
            #Começamos verificando qual o ponto de saida
            #do veículo 'k' e atribuímos esse nodo para
            #o ponto de partida do nosso loop

            actual_node = routes[k].pop(0)

            #inicial->actual_node = primeiro arco de todas as rotas

            #Iremos percorrer a rota até que alcançemos a vítima
            while actual_node !=0:
                # actual_node                actual_node
                #vira chegada do        inicia como saída do
                #  arco                        arco
                #   |                            |
                #   v                            v
                actual_node = routes[k].pop(actual_node)

                #it0: actual_node-> nodo 'j'
                #               ^
                #               |
                #it1:     actual_node -> nodo 'k'
                #                             ^
                #                             |
                #it2:                     actual_node -> nodo 'l' ...


        #Se sobrou algum subciclo
        #depois da remoção do caminho
        #principal então 
        #len(routes[k]) > 0
        summ += len(routes[k])


        #Se sobrou algum arco dentro
        #da rota, então deve ser algum
        #subciclo que devemos retirar
        if len(routes[k]) >0:

            #Enquanto ainda existir algum arco
            #dentro da rota do veículo 'k'
            #ele faz parte de um subciclo
            #que deve ser retirado, portanto
            #iteramos até removermos todos os
            #subciclos
            while(len(routes[k])>0):
                print(routes[k])
                cut = LinearExpr(model) #Expressão Linear do Subciclo atual

                first_node = list(routes[k].keys())[0]
                #  ^
                #  |
                #saída do primeiro arco
                #do subciclo atual
                #
                #chegada do primeiro
                #arco do subciclo atual
                #  |
                #  v
                actual_node = routes[k].pop(first_node)

                #Adicionamos o arco inicial à restrição
                cut += travels[k,int(first_node),int(actual_node)]

                #Tamanho do subciclo -1
                size = 0
                while(actual_node != first_node):
                    next_node = routes[k].pop(actual_node)                 #Percorre o subciclo e o adiciona ao#
                    cut += travels[k,int(actual_node),int(next_node)]#conjunto de subciclos proíbidos.#
                    actual_node = next_node
                    size+=1
                count+=1
                model.add_constraint(cut<=size,"SubCycleCut_"+str(count))

    if summ == 0:
        subcicle = False
print(time.time()-start)

Version identifier: 22.1.0.0 | 2022-03-09 | 1a383f8ce
CPXPARAM_Read_DataCheck                          1
Tried aggregator 1 time.
MIP Presolve eliminated 4751 rows and 3550 columns.
MIP Presolve modified 2026 coefficients.
Reduced MIP has 569 rows, 6950 columns, and 33325 nonzeros.
Reduced MIP has 6950 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (29.11 ticks)
Found incumbent of value 608.000000 after 0.04 sec. (54.41 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 569 rows, 6950 columns, and 33325 nonzeros.
Reduced MIP has 6950 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (21.81 ticks)
Probing time = 0.02 sec. (8.23 ticks)
Clique table members: 20219.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.06 sec. (42.01 ticks)

        Nodes                                         Cuts/
  

      0     2      120.0000    43      155.0000      120.0000      753   22.58%
Elapsed time = 1.26 sec. (1014.05 ticks, tree = 0.02 MB, solutions = 14)
*    10+    4                          154.0000      120.0000            22.08%
     74    59      140.6364    28      154.0000      120.0000     4572   22.08%
*    83    49      integral     0      153.0000      120.0000     3935   21.57%
*   130    89      integral     0      151.0000      120.0000     7072   20.53%
    402   194        cutoff            151.0000      131.8056    15607   12.71%
    631   343      140.9455    51      151.0000      132.1429    24536   12.49%
    928   588      140.3288    13      151.0000      132.1429    40046   12.49%
   1201   702        cutoff            151.0000      134.2732    50672   11.08%
   1318   798      139.2368    27      151.0000      134.4785    61366   10.94%
   1610   944      143.5036    36      151.0000      135.4870    75212   10.27%
   1944  1181      142.4781    38      151.0000

   3801   174      134.3167    83      151.0000      134.9978   167305   10.60%
   4453   473      138.5263    35      151.0000      134.9978   203236   10.60%
   5304   872        cutoff            151.0000      137.2382   256218    9.11%
   5878   963      147.6667    11      151.0000      139.7373   300844    7.46%
   6634   821        cutoff            151.0000      141.9224   348846    6.01%
   8094   273    infeasible            151.0000      147.1053   394198    2.58%

GUB cover cuts applied:  3
Cover cuts applied:  137
Flow cuts applied:  1
Mixed integer rounding cuts applied:  3
Zero-half cuts applied:  1
Gomory fractional cuts applied:  1

Root node processing (before b&c):
  Real time             =    1.90 sec. (1622.84 ticks)
Parallel b&c, 8 threads:
  Real time             =   13.69 sec. (10903.31 ticks)
  Sync time (average)   =    2.32 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =   15.59 sec. (12526.15 ticks

Probing time = 0.02 sec. (8.25 ticks)
Clique table members: 20227.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.03 sec. (27.78 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                          153.0000        0.0000           100.00%
      0     0      120.0000    54      153.0000      120.0000      360   21.57%
      0     0      120.0000    70      153.0000       Cuts: 2      420   21.57%
*     0+    0                          151.0000      120.0000            20.53%
      0     0      120.0000    70      151.0000       Cuts: 2      535   20.53%
Repair heuristic found nothing.

Repeating presolve.
Tried aggregator 2 times.
MIP Presolve eliminated 7 rows and 2092 columns.
MIP Presolve modified 857 coefficients.
Aggregator did 3 substitutions

Elapsed time = 1.91 sec. (1606.07 ticks, tree = 0.02 MB, solutions = 14)
     60    45      139.1316    21      151.0000      120.0000     4389   20.53%
    335   180      136.2161    27      151.0000      120.0000    13517   20.53%
    674   413      139.9123    14      151.0000      120.0000    27852   20.53%
    857   609      134.0910    32      151.0000      131.6776    40011   12.80%
   1281   804      135.2340    38      151.0000      131.6776    47407   12.80%
   1634  1189      138.2212    50      151.0000      131.6776    63503   12.80%
   1966  1444      137.3588    16      151.0000      132.1667    76536   12.47%
   2135  1681      137.1765    20      151.0000      132.1667    90467   12.47%
   2531  1865      139.4656    44      151.0000      133.4870    99147   11.60%

Performing restart 1

Repeating presolve.
Tried aggregator 1 time.
MIP Presolve eliminated 15 rows and 168 columns.
MIP Presolve modified 8275 coefficients.
Reduced MIP has 550 rows, 4475 columns, and 18017

...

Retaining values of one MIP start for possible repair.
Tried aggregator 1 time.
MIP Presolve eliminated 4751 rows and 3550 columns.
MIP Presolve modified 2026 coefficients.
Reduced MIP has 691 rows, 6950 columns, and 33619 nonzeros.
Reduced MIP has 6950 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (29.75 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 691 rows, 6950 columns, and 33619 nonzeros.
Reduced MIP has 6950 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (21.76 ticks)
Probing time = 0.02 sec. (8.24 ticks)
Clique table members: 20366.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.04 sec. (62.99 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                         


        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                          152.0000      120.0000            21.05%
      0     0      120.0000    73      152.0000      120.0000     1613   21.05%
Repair heuristic found nothing.
      0     2      120.0000    37      152.0000      120.0000     1613   21.05%
Elapsed time = 1.96 sec. (1744.77 ticks, tree = 0.02 MB, solutions = 9)
     28     8      135.1821    19      152.0000      120.0000     2654   21.05%
    235   107        cutoff            152.0000      120.0000    12118   21.05%
    572   351      137.7502    20      152.0000      120.0000    26832   21.05%
    947   611      148.2643    56      152.0000      120.0000    35994   21.05%
   1135   847      132.5552    54      152.0000      120.0000    44197   21.05%
   1429  1126      135.9737    90      152.0000      120.0000    64068   21.05%
   1713  1326      134.3570    87  

MIP Presolve modified 8750 coefficients.
Reduced MIP has 667 rows, 4480 columns, and 18562 nonzeros.
Reduced MIP has 4480 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (16.04 ticks)
Tried aggregator 1 time.
Reduced MIP has 667 rows, 4480 columns, and 18562 nonzeros.
Reduced MIP has 4480 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (12.89 ticks)
Represolve time = 0.12 sec. (105.42 ticks)
   7800     0      120.0000    67      152.0000       Cuts: 8   264000   12.51%
   7800     0      120.0000    74      152.0000      Cuts: 20   264135   12.51%
   7800     2      120.0000    36      152.0000      132.9799   264135   12.51%
Elapsed time = 28.57 sec. (33394.52 ticks, tree = 0.02 MB, solutions = 9)
   8487   235      136.8552    32      152.0000      132.9799   281764   12.51%
   9277   920      137.0565    81      152.0000      132.9799   311440   12.51%
  10386  1732      150.0000     6      152.0000      132.9799   352511   12.51%

  96900 53043      147.6285    77      152.0000      140.5111  3921113    7.56%
  99062 53544      145.4545    34      152.0000      140.6447  4023357    7.47%
 100541 54513        cutoff            152.0000      140.7775  4181393    7.38%
 103100 55316      147.0222    43      152.0000      140.9097  4338552    7.30%
Elapsed time = 166.16 sec. (151200.13 ticks, tree = 331.96 MB, solutions = 9)
 105670 55861        cutoff            152.0000      141.0199  4433802    7.22%
 106906 56292      144.8344    41      152.0000      141.1812  4517972    7.12%
 108658 56706      143.4903    46      152.0000      141.3155  4647825    7.03%
 111051 56954      149.1968    46      152.0000      141.4379  4722879    6.95%
 113187 57358      149.5152    32      152.0000      141.5848  4828275    6.85%
 114353 57822      149.9584    53      152.0000      141.6915  4956534    6.78%
 117299 58066        cutoff            152.0000      141.8363  5039765    6.69%
 117963 58384      148.9095    54      152

Presolve time = 0.02 sec. (29.78 ticks)
Tried aggregator 1 time.
Detecting symmetries...
Reduced MIP has 694 rows, 6950 columns, and 33625 nonzeros.
Reduced MIP has 6950 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.02 sec. (21.84 ticks)
Probing time = 0.02 sec. (8.27 ticks)
Clique table members: 20369.
MIP emphasis: balance optimality and feasibility.
MIP search method: dynamic search.
Parallel mode: deterministic, using up to 8 threads.
Root relaxation solution time = 0.05 sec. (55.27 ticks)

        Nodes                                         Cuts/
   Node  Left     Objective  IInf  Best Integer    Best Bound    ItCnt     Gap

*     0+    0                          154.0000        0.0000           100.00%
      0     0      120.0000    53      154.0000      120.0000      665   22.08%
*     0+    0                          152.0000      120.0000            21.05%
Repair heuristic found nothing.

Repeating presolve.
Tried aggregator 2 times.
MIP Presolve eliminat

In [20]:
model.export_as_lp("model")

'model.lp'